In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from datetime import date
import math
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import joblib
import pickle
import plotly.graph_objects as go

ModuleNotFoundError: No module named 'yfinance'

In [ ]:
START = "2010-01-01"
END = date.today().strftime("%Y-%m-%d")

In [ ]:
def load_data(ticker):
    data = yf.download(ticker,START,END)
    data.reset_index(inplace=True)
    return data

In [ ]:
df = load_data('AAPL')

In [ ]:
df = df.drop('Adj Close',axis=1)

In [ ]:
df_Close = df[['Close','Date']]

In [ ]:
df_Close.set_index('Date')

In [ ]:
df_open = df[['Open','Date']]

In [ ]:
#Filter Close column insted of deleting it
df_open = df_open.drop('Date',axis=1

In [ ]:
df_open.shape

In [ ]:
#Save values in an array
df_open

In [ ]:
traindata_size = math.ceil(len(df_open)*.8)
testdata_size = len(df_open) - traindata_size

In [ ]:
traindata_size

In [ ]:
testdata_size

In [ ]:
train = df_open[0:traindata_size]
print(len(train))

In [ ]:
sc = MinMaxScaler(feature_range = (0,1))
scaled_train = sc.fit_transform(train)

In [ ]:
X_train = []
y_train = []

for i in range(60, len(scaled_train)):
    X_train.append(scaled_train[i-60:i, 0])
    y_train.append(scaled_train[i, 0])
    

X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))

In [ ]:
model = keras.Sequential()

model.add(layers.LSTM(units= 50, return_sequences = True, input_shape= (X_train.shape[1],1)))
model.add(layers.Dropout(0.2))

model.add(layers.LSTM(units= 50, return_sequences = True))
model.add(layers.Dropout(0.2))

model.add(layers.LSTM(units= 50, return_sequences = True))
model.add(layers.Dropout(0.2))

model.add(layers.LSTM(units= 25))
model.add(layers.Dropout(0.2))

model.add(layers.Dense(units = 1))

In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:

model.fit(X_train,y_train, batch_size = 1, epochs = 5)

In [ ]:
test = df_open[traindata_size - 60:]
print(len(test))

In [ ]:
scaled_test = sc.fit_transform(test)

In [ ]:
X_test = []
y_test = df_open[traindata_size:]
for i in range(60,len(scaled_test)):
    X_test.append(scaled_test[i-60:i, 0])

X_test = np.array(X_test)

In [ ]:
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))

In [ ]:

pred = model.predict(X_test)rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse
predictions = sc.inverse_transform(pred)

In [ ]:
train = df_open[:traindata_size]
valid = df_open[traindata_size:]
valid['predictions'] = predictions
plt.figure(figsize=(16,8))
plt.title("Time series forecast with LSTM")
plt.xlabel("Days")
plt.ylabel("Closing price in USD($)")
plt.plot(train['Open'])
plt.plot(valid[['Open','predictions']])
plt.legend(['Train','Val','Predicted'],loc='lower right')
plt.show()

In [ ]:
predictions[570]

In [ ]:
valid

In [ ]:
model.save('LSTM2.h5')